# Cours d'introduction à la technologie Chipwhisperer 

## Correlation Power Analysis : l'arme ultime

Jeune padawan, si vous êtes arrivé jusqu'ici, alors c'est que vous êtes digne de découvrir l'arme ultime des Side Channel Attacks sur les systèmes non protégés : les attaques par CPA, ou **Correlation Power Analysis**.  \
Tout d'abord, elles nécessitent en moyenne **moins de traces** que les attaques DPA pour reconstituer la clé secrète, grâce à l’usage du coefficient de corrélation qui extrait plus efficacement l’information du bruit des traces de consommation. Deuxièmement, elles sont **plus robustes** car en remplaçant la simple différence de moyennes par le coefficient de corrélation de Pearson, elles deviennent moins dépendantes du choix précis d’un point d’intérêt ou d’un seuil de discrimination. Et enfin, elles supportent mieux les variations d’alignement et le bruit, et permettent d’intégrer des modèles de fuite multi-bit (Hamming weight, T-Tables 32 bits, etc.).

Mais quels sont leurs fondements mathématiques ? Et en quoi consistent-elles exactement ? \
Les attaques CPA reposent sur le choix d'un modèle de fuite et d'un modèle de consommation. Le premier postule qu’à un instant critique (par exemple juste après une opération de S-Box), la consommation instantanée $T_i(t)$ du système sur la i-ème exécution peut s’écrire :
$$ T_i(t) = L_i(t) + N_i(t) $$, \
avec $L_i(t)$ la composante de fuite dépendante des données traitées (et donc de la clé), et $N_i(t)$ le bruit associé. \
Le second quant à lui qui permet de simuler la consommation électrique réelle $T_i(t)$ qu'aurait produit le traitement de certaines données par le système, généralement en utilisant le poids ou la distance de Hamming. 

Ainsi, pour chaque hypothèse de clé $k$, on construit un vecteur correspondant à la consommation hypothétique qu'aurait produit cette clé si elle avait été utilisée par le système : $$h_i(k) = HW(V_i(k))$$, avec $V_i(k) = f(P_i, k)$ une valeur intermédiaire de l'algorithme cryptographique obtenue à partie de la clé hypothétique $k$ et d'un texte clair $P_i$.

On recueille en parallèle $N$ traces réelles de consommation électrique $X = {\{T_i\}}_{i=1,..,N}$.

Ensuite, on calcule le composant essentiel de cette attaque : le coefficient de corrélation de Pearson, à l'aide de la formule suivante : $$r = \frac{cov(X, Y)}{\sigma_X \sigma_Y}$$, où $$cov(X, Y) = \sum_{n=1}^{N}[(Y_n - \bar{Y})(X_n - \bar{X})]$$ et $$\sigma_X = \sqrt{\sum_{n=1}^{N}(X_n - \bar{X})^2}$$. Nous cherchons ainsi à déterminer la corrélation entre une **trace de consommation** et le **poids de Hamming d'une hypothèse de clé**.

Les étapes de l'attaque sont donc les suivantes : 
- recueillir nos **traces de consommation réelles** à partir du modèle de fuite 
- pour chaque hypothèse de clé, calculer la **consommation hypothétique associée** à l'aide du modèle de consommation choisi
- pour chaque hypothèse de clé, calculer la **corrélation entre la consommation hypothétique associée et la consommation réelle mesurée**
- la corrélation **la plus élevée** correspond à la bonne valeur de clé secrète

## CPA sur AES

Commençons par recueillir nos traces et réutilisons les outils développés dans le lab précédent sur DPA en lien avec AES :

In [4]:
from cwtraces import sca101_lab_data
import chipwhisperer as cw

data = sca101_lab_data["lab4_2"]()
trace_array =  data["trace_array"]
textin_array = data["textin_array"]
key = data["key"]

assert len(trace_array) == 50
print("✔️ OK to continue!")

✔️ OK to continue!


In [5]:

sbox = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def aes_internal(inputdata, key):
    return sbox[inputdata ^ key]

# compte le nombre de 1 = poids de Hamming de n
def calc_hamming_weight(n):
    return bin(n).count("1")

HW = []
for num in range(0,256):
    HW.append(calc_hamming_weight(num))

La prochaine étape est maintenant d'implémenter les formules mathématiques vues précédemment à l'aide du module Numpy : 

In [10]:
import numpy as np

def mean(X): # moyenne
    return np.sum(X, axis=0)/len(X)

def std_dev(X, X_bar): # écart type avec X_bar la moyenne des traces de X
    return np.sqrt(np.sum(np.power(X - X_bar, 2), axis=0))
    
def cov(X, X_bar, Y, Y_bar): # covariance, attention axis=0 -> somme chaque trace en chaque point
    return np.sum((Y - Y_bar)*(X - X_bar), axis=0)

Nous pouvons maintenant passer au corps de l'attaque en elle-même. Rappelons-le, le coefficient de Pearson est calculé avec X = traces de consommation réelles capturées (`trace_array` dans notre cas) et Y = consommation hypothétique = `hamming_weigth(résultat S_box(XOR) avec hypothèse de clé)` = `aes_internal(input_data, key_byte_guess)`. Ensuite, l'étape suivante est de déterminer l'hypothèse de clé produisant la corrélation maximale, puisqu'il s'agira de notre guess 'correct' pour la vraie valeur de clé secrète utilisée par le système. Implémentons tout ceci : 

In [11]:
from tqdm.notebook import trange

maxcpa = [0] * 256

t_bar = mean(trace_array) # calculates the mean and standard_deviation for a entire trace at once
o_t = std_dev(trace_array, t_bar)

for kguess in trange(0, 256): # for each possible key byte value
    # hamming weigth of the hypothetical leakage for that key guess
    hws = np.array([[HW[aes_internal(textin[0],kguess)] for textin in textin_array]]).transpose()
    
    hws_bar = mean(hws)
    # correlation coefficient calculation (it's the exact same formula as the one we saw above)
    cpaoutput = cov(trace_array, t_bar, hws, hws_bar) / (std_dev(trace_array, t_bar) * std_dev(hws, hws_bar))
    # we store the maximum correlation value
    maxcpa[kguess] = max(abs(cpaoutput))

# get the key guess with the maximum correlation value
guess = np.argmax(maxcpa)
guess_corr = max(maxcpa)

print("Key guess: ", hex(guess))
print("Correlation: ", guess_corr)

  0%|          | 0/256 [00:00<?, ?it/s]

Key guess:  0x2b
Correlation:  0.8270952071731353


Pour reconstituer la clé entière, il suffit de répéter cette attaque pour chacun de ses bytes en englobant le texte précédant dans une boucle `for byte_number in range(0, 16)` et en prenant garde à utiliser le bon byte du texte clair ! 

### Attaque CPA sur AES avec l'outil Chipwhisperer Analyzer (ou l'industrialisation de l'armement version side channels)

S'il est important de réaliser les attaques ci-dessus 'à la main' pour bien en comprendre les rouages, dans la pratique il ne serait pas réaliste d'espérer toujours les faire de cette façon. L'objectif de l'outil **Chipwhisperer Analyzer**, l’interface logicielle principale du Chipwhisperer, est ainsi d'automatiser les attaques sca sur des systèmes cibles, et en particulier les attaques CPA. \
Nous n'avons donc plus besoin de la `S-Box`rédigée explicitement, ni même de nos `Numpy arrays`pour stocker les traces : tout est stocké dans le Chipwhisperer Analyzer, et nous utiliserons maintenant des Chipwhisperer projects pour récupérer nos traces de consommation.

In [12]:
from cwtraces import sca101_lab_data
import chipwhisperer as cw

data = sca101_lab_data["lab4_3"]
proj = data["project"]()

L'accès au Chipwhisperer Analyzer se fait via la commande suivante : 

In [13]:
import chipwhisperer.analyzer as cwa

Ensuite, nous devons spécifier le modèle de fuite requis pour l'attaque, ici la sortie de `S-Box`AES : 

In [14]:
leak_model = cwa.leakage_models.sbox_output

Et enfin on configure l'Analyzer pour une attaque CPA en une ligne : 

In [16]:
attack = cwa.cpa(proj, leak_model)
print(attack)

project     = <chipwhisperer.common.api.ProjectFormat.Project object at 0x11753bd90>
leak_model  = <chipwhisperer.analyzer.attacks.models.AES128_8bit.AES128_8bit object at 0x1175d99a0>
algorithm   = <chipwhisperer.analyzer.attacks.cpa_algorithms.progressive.CPAProgressive object at 0x1175fc850>
trace_range = [0, 50]
point_range = [0, 5000]
subkey_list = range(0, 16)



Le lancement de l'attaque se fait également en une seule ligne : 

In [17]:
results = attack.run()
print(results)

Subkey KGuess Correlation
  00    0x2B    0.81091
  01    0x7E    0.79538
  02    0x15    0.87392
  03    0x16    0.80839
  04    0x28    0.83281
  05    0xAE    0.87081
  06    0xD2    0.89009
  07    0xA6    0.85979
  08    0xAB    0.82772
  09    0xF7    0.80872
  10    0x15    0.85116
  11    0x88    0.77970
  12    0x09    0.81269
  13    0xCF    0.92473
  14    0x4F    0.80885
  15    0x3C    0.80804



Pour obtenir plus d'informations à propos des résultats obtenus lors de l'attaque, plusieurs pistes d'exploration sont possibles : 

- afficher les corrélations maximales obtenues pour chaque hypothèse de clé et les stocker dans une **structure de données complexe**

In [18]:
import pandas as pd
stat_data = results.find_maximums()
df = pd.DataFrame(stat_data).transpose()
print(df.head())

                                0                                1   \
0   [43, 1948, 0.8109063524779131]  [126, 2145, 0.7953842364446794]   
1  [252, 2884, 0.6423479963383932]   [38, 4285, 0.6613321299284041]   
2  [141, 1790, 0.6296736535214246]    [51, 4470, 0.619074118857893]   
3  [213, 2929, 0.6119273120624024]  [198, 3996, 0.6075096829196474]   
4  [255, 4959, 0.5994112600421648]   [21, 4457, 0.6030513983381995]   

                                2                                3   \
0   [21, 2340, 0.8739238610740697]   [22, 2537, 0.8083871957141481]   
1  [194, 1968, 0.6502311623652279]  [248, 1284, 0.6225485579163301]   
2    [88, 1010, 0.606686404246976]  [234, 1448, 0.6092365492443333]   
3  [116, 2303, 0.6057832525002114]   [64, 3909, 0.6005843601058458]   
4    [61, 908, 0.6019273799935005]  [222, 2086, 0.5991698177329257]   

                               4                                5   \
0  [40, 1993, 0.8328122975012535]  [174, 2188, 0.8708071759128995]   
1  [22

- afficher les meilleures corrélations **au fur et à mesure** de l'attaque, et pas seulement à la fin comme dans l'attaque conduite précédemment (possible car ChipWhisperer Analyzer utilise en fait un calcul de corrélation « en ligne », i.e pour chaque point)

In [22]:
from IPython.display import clear_output # to clear the table

key = proj.keys[0]

def format_stat(stat): # pour formater les corrélations en sortie
    return str("{:02X}<br>{:.3f}".format(stat[0], stat[2]))

def color_corr_key(row): # préférences d'affichage (corrélation maximale en rouge)
    global key
    ret = [""] * 16
    for i,bnum in enumerate(row):
        if bnum[0] == key[i]:
            ret[i] = "color: red"
        else:
            ret[i] = ""
    return ret
    
def stats_callback(): # fonction formatage + affichage des résultats de l'attaque appelée périodiquement lors de l'attaque
    results = attack.results
    results.set_known_key(key)
    stat_data = results.find_maximums()
    df = pd.DataFrame(stat_data).transpose()
    clear_output(wait=True)
    display(df.head().style.format(format_stat).apply(color_corr_key,axis=1))
    
results = attack.run(stats_callback, 10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,2B0.811,7E0.795,150.874,160.808,280.833,AE0.871,D20.890,A60.860,AB0.828,F70.809,150.851,880.780,090.813,CF0.925,4F0.809,3C0.808
1,FC0.642,260.661,C20.650,F80.623,DD0.657,F30.651,990.660,520.617,B00.639,7F0.626,0D0.631,E50.668,B60.644,140.633,2A0.620,EF0.614
2,8D0.630,330.619,580.607,EA0.609,300.632,130.637,B60.640,7A0.608,AA0.616,5D0.599,E90.608,F50.617,460.627,2B0.611,AD0.611,640.607
3,D50.612,C60.608,740.606,400.601,BF0.607,790.631,E90.639,2D0.602,E70.606,040.599,5E0.608,320.614,DD0.609,590.608,F10.602,850.600
4,FF0.599,150.603,3D0.602,DE0.599,330.596,D50.613,C70.632,910.601,0B0.592,BE0.596,CD0.606,BB0.611,520.608,240.605,E60.599,100.599


Si le code précédent vous effraie, une fonction de callback Jupyter par défaut est également disponible :

In [24]:
import chipwhisperer as cw
cb = cwa.get_jupyter_callback(attack)
results = attack.run(cb, 10)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
PGE=,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,2B0.811,7E0.795,150.874,160.808,280.833,AE0.871,D20.890,A60.860,AB0.828,F70.809,150.851,880.780,090.813,CF0.925,4F0.809,3C0.808
1,FC0.642,260.661,C20.650,F80.623,DD0.657,F30.651,990.660,520.617,B00.639,7F0.626,0D0.631,E50.668,B60.644,140.633,2A0.620,EF0.614
2,8D0.630,330.619,580.607,EA0.609,300.632,130.637,B60.640,7A0.608,AA0.616,5D0.599,E90.608,F50.617,460.627,2B0.611,AD0.611,640.607
3,D50.612,C60.608,740.606,400.601,BF0.607,790.631,E90.639,2D0.602,E70.606,040.599,5E0.608,320.614,DD0.609,590.608,F10.602,850.600
4,FF0.599,150.603,3D0.602,DE0.599,330.596,D50.613,C70.632,910.601,0B0.592,BE0.596,CD0.606,BB0.611,520.608,240.605,E60.599,100.599


In [ ]:
Cette section souligne la puissance des attaques CPA sur des systèmes non protégés par des contre-mesures comment, 

## Conclusion

C'est ici que se termine notre bout de chemin ensemble, mais la route est encore longue si vous souhaitez découvrir la richesse et la complexité de ce domaine de recherche très actif que sont les attaques par canaux auxiliaires et, qui sait, peut-être même pourrez vous ajouter votre pierre à l'édifice !